## **Scatter Plot**

### **Librerías**

In [1]:
import pandas as pd
import polars as pl

In [2]:
import plotly.express as px
import plotly.graph_objects as go

### **Datos**

In [3]:
df = pl.read_csv('../../Data/Raw/Pokemon.csv')

In [4]:
df.sample(5)

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
455,"""Carnivine""","""Grass""",null,454,74,100,72,90,72,46,4,false
386,"""DeoxysSpeed Forme""","""Psychic""",null,600,50,95,90,95,90,180,3,true
130,"""Gyarados""","""Water""","""Flying""",540,95,125,79,60,100,81,1,false
102,"""Exeggcute""","""Grass""","""Psychic""",325,60,40,80,60,45,40,1,false
708,"""Phantump""","""Ghost""","""Grass""",309,43,70,48,50,60,38,6,false


### **Limpieza**

In [5]:
df.columns = [col.replace('.', '').replace(' ', '_') for col in df.columns]

In [50]:
df_gen_1 = df.with_columns(
    pl.when(pl.col('Legendary') == True)
        .then(pl.lit('Legendary'))
        .otherwise(pl.lit('Non-Legendary'))
        .alias('Legendary_Type')
)

In [51]:
df_gen_1 = df_gen_1.filter(
    (pl.col('Generation') == 1) & 
    (~pl.col('Name').str.contains('Mega'))
).to_pandas()

In [ ]:
avg_attack = df_gen_1['Attack'].mean()
avg_defense = df_gen_1['Defense'].mean()

### **Visualizaciones**

#### **Scatterplot - Basic**

In [52]:
px.scatter(
    data_frame=df_gen_1,
    x='Attack',
    y='Defense',
    color='Type_1',
    size='HP',
    hover_name = 'Name',
    symbol='Legendary_Type',
    symbol_map={
        'Legendary': 'star',
        'Non-Legendary': 'circle'
    }
)

#### **Scatterplot - Layout**

In [53]:
fig = px.scatter(
    data_frame=df_gen_1,
    x='Attack',
    y='Defense',
    color='Type_1',
    size='HP',
    hover_name = 'Name',
    symbol='Legendary_Type',
    symbol_map={
        'Legendary': 'star',
        'Non-Legendary': 'circle'
    }
)

fig.update_layout(
    title = 'Relación entre Ataque y Defensa en Pokémon de la Generación 1',
    xaxis_title = 'Ataque',
    yaxis_title = 'Defensa',
    legend_title = 'Tipo de Pokémon',
    font = dict(size=13)
)

fig.show()

#### **Scatterplot - Lineas de Referencia**

In [67]:
fig = px.scatter(
    data_frame=df.filter(
        (pl.col('Generation') == 1) & 
        (~pl.col('Name').str.contains('Mega'))
    ).to_pandas(),
    x='Attack',
    y='Defense',
    color='Type_1',
    size='HP',
    hover_name = 'Name',
    symbol='Legendary_Type',
    symbol_map={
        'Legendary': 'star',
        'Non-Legendary': 'circle'
    }
)

fig.add_shape(
    type='line',
    name='Avg_attack',
    x0=avg_attack,
    x1=avg_attack,
    y0=0,
    y1=200,
    line=dict(
        color='red',
        width=1,
        dash='dash'
    )
)

fig.add_shape(
    type='line',
    name='Avg_attack',
    x0=0,
    x1=200,
    y0=avg_defense,
    y1=avg_defense,
    line=dict(
        color='green',
        width=1,
        dash='dash'
    )
)


fig.update_layout(
    title = 'Relación entre Ataque y Defensa en Pokémon de la Generación 1',
    xaxis_title = 'Ataque',
    yaxis_title = 'Defensa',
    legend_title = 'Tipo de Pokémon',
    font = dict(size=13)
)

fig.show()